In [1]:
import import_ipynb

In [2]:
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'

import random, datetime
from pathlib import Path

import gymnasium
from gymnasium.wrappers import FrameStack, GrayScaleObservation, TransformObservation
from metrics import MetricLogger
from agent import SpaceInvader
from wrappers import ResizeObservation, SkipFrame

importing Jupyter notebook from metrics.ipynb
importing Jupyter notebook from agent.ipynb
importing Jupyter notebook from neural.ipynb
importing Jupyter notebook from wrappers.ipynb


In [3]:
import gymnasium as gym

In [4]:
class SkipFrame(gym.Wrapper):
    def __init__(self, env, skip):
        """Return only every `skip`-th frame"""
        super().__init__(env)
        self._skip = skip

    def step(self, action):
        """Repeat action, and sum reward"""
        total_reward = 0.0
        done = False
        for i in range(self._skip):
            # Accumulate reward and repeat the same action
            obs, reward, done, truncated, info = self.env.step(action)
            total_reward += reward
            if done:
                break
        return obs, total_reward, done, truncated, info

In [5]:
env = gymnasium.make("ALE/SpaceInvaders-v5", mode=2)

A.L.E: Arcade Learning Environment (version 0.8.1+53f58b7)
[Powered by Stella]


In [6]:
env = SkipFrame(env, skip=4)
env = GrayScaleObservation(env, keep_dim=False)
env = ResizeObservation(env, shape=84)
env = TransformObservation(env, f=lambda x: x / 255.)
env = FrameStack(env, num_stack=4)

In [7]:
env.reset()

(<gymnasium.wrappers.frame_stack.LazyFrames at 0x284ab70b0>,
 {'lives': 3, 'episode_frame_number': 3, 'frame_number': 3})

In [8]:
save_dir = Path('checkpoints') / datetime.datetime.now().strftime('%Y-%m-%dT%H-%M-%S')
save_dir.mkdir(parents=True)

In [9]:
env.action_space.n

6

In [10]:
checkpoint = None
spaceInvader = SpaceInvader(state_dim=(4,84,84), action_dim=env.action_space.n, save_dir=save_dir, checkpoint=checkpoint)

In [11]:
logger = MetricLogger(save_dir)


In [12]:
episodes = 40000

In [13]:
action = 1

In [14]:
result = env.step(action)
next_state = result[0]
reward = result[1]
done = result[2]
info = result[3]

/Users/tobiaspoulsen/Library/Caches/pypoetry/virtualenvs/tobiaspoulsen--jf9GMjB-py3.11/lib/python3.11/site-packages/gymnasium/utils/passive_env_checker.py:249: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):


In [15]:
result = env.step(action)
result

(<gymnasium.wrappers.frame_stack.LazyFrames at 0x28971ebb0>,
 0.0,
 False,
 False,
 {'lives': 3, 'episode_frame_number': 35, 'frame_number': 35})

In [16]:
state = env.reset()
state

(<gymnasium.wrappers.frame_stack.LazyFrames at 0x2b0c5af70>,
 {'lives': 3, 'episode_frame_number': 3, 'frame_number': 38})

In [17]:
### for Loop that train the model num_episodes times by playing the game
for e in range(episodes):

    state = env.reset()
    state = state[0]
    # Play the game!
    while True:

        # 3. Show environment (the visual) [WIP]
        # env.render()

        # 4. Run agent on the state
        action = spaceInvader.act(state)
        # 5. Agent performs action
        result = env.step(action)
        next_state = result[0]
        reward = result[1]
        done = result[2]
        info = result[3]
        
        # 6. Remember
        spaceInvader.cache(state, next_state, action, reward, done)

        # 7. Learn
        q, loss = spaceInvader.learn()

        # 8. Logging
        logger.log_step(reward, loss, q)

        # 9. Update state
        state = next_state
        # 10. Check if end of game
        if done or info:
            break
    

    logger.log_episode()

    if e % 20 == 0:
        logger.record(
            episode=e,
            epsilon=spaceInvader.exploration_rate,
            step=spaceInvader.curr_step
        )

<string>:73: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/torch/csrc/utils/tensor_new.cpp:248.)


Episode 0 - Step 138 - Epsilon 0.9999655005908009 - Mean Reward 135.0 - Mean Length 138.0 - Mean Loss 0.0 - Mean Q Value 0.0 - Time Delta 1.565 - Time 2023-05-11T19:11:48
Episode 20 - Step 2733 - Epsilon 0.9993169832766863 - Mean Reward 135.714 - Mean Length 130.143 - Mean Loss 0.0 - Mean Q Value 0.0 - Time Delta 13.995 - Time 2023-05-11T19:12:02
Episode 40 - Step 5723 - Epsilon 0.9985702728560336 - Mean Reward 159.024 - Mean Length 139.585 - Mean Loss 0.0 - Mean Q Value 0.0 - Time Delta 16.453 - Time 2023-05-11T19:12:18
Episode 60 - Step 8263 - Epsilon 0.9979363819351096 - Mean Reward 150.656 - Mean Length 135.459 - Mean Loss 0.0 - Mean Q Value 0.0 - Time Delta 14.207 - Time 2023-05-11T19:12:32
Episode 80 - Step 10729 - Epsilon 0.9973213436837014 - Mean Reward 143.21 - Mean Length 132.457 - Mean Loss 0.0 - Mean Q Value 0.0 - Time Delta 14.067 - Time 2023-05-11T19:12:47
Episode 100 - Step 13513 - Epsilon 0.9966274494448698 - Mean Reward 146.65 - Mean Length 133.75 - Mean Loss 0.0 - Mea

<Figure size 640x480 with 0 Axes>

NameError: name 's' is not defined